# 3D Magnetized TOV Test

This notebook runs a simulation of magnetized TOV in fixed spacetime with AsterX, and visualizes the data saved in OpenPMD format. 


### Test setup

- Uses `TOVSolver` thorn to compute the initial TOV star configuration using a polytropic EOS with adiabatic index $\gamma=2.0$, polytropic constant $K=100$ and initial central rest-mass density $\rho=1.28\times 10^{-3}$.


- Initial vector potential is set using the thorn ```AsterSeeds```: $A_\phi \equiv A_b r^2 max(p-p_{cut}, 0)^{n_s}$, where $p_{cut} = 0.04p_{max}$ and $n_s=2$. This gives an initial magnetic field strength of about $10^{15} G$.

- Evolution with ideal gas EOS with $\gamma = 2$.

- Grid domain is $[-13,+13]$ with size $26 \times 26 \times 26$ (very low resolution).

- `Neumann` boundary conditions are applied in all directions.




In [ ]:
# this allows you to use "cd" in cells to change directories instead of requiring "%cd"
%automagic on
# override IPython's default %%bash to not buffer all output
from IPython.core.magic import register_cell_magic
@register_cell_magic
def bash(line, cell): get_ipython().system(cell)

# this (non-default package) keeps the end of shell output in view
try: import scrolldown
except ModuleNotFoundError: pass

# set Python environment 
import os, sys
os.environ["PYTHONUSERBASE"] = os.environ['HOME'] + "/Cactus/python"
sys.path.insert(1, f"{os.environ['PYTHONUSERBASE']}/lib/python{sys.version_info[0]}.{sys.version_info[1]}/site-packages")

## 1. Steps to perform the simulation

First, let's first move to the Cactus folder:

In [ ]:
cd ~/CarpetX

Now, let's create the parameter file to be used for this simulation:

In [ ]:
%%writefile ./par/MagnetizedTOV.par

###############################
# Simple test of magnetised TOV neutron star
# Same neutron star as the gallery example
# K=100, rho_c = 1.28e-3 => M = 1.4, M_b = 1.506
# evolve for t = 400 M
##############################
ActiveThorns = "
    ADMBase
    CarpetX
    HydroBase
    IOUtil
    ODESolvers
    SystemTopology
    TimerReport
    TmunuBase
    AsterX
    AsterSeeds
"

$nlevels        = 1
$ncells         = 26

Cactus::terminate = "time"
Cactus::cctk_final_time = 400

CarpetX::verbose = no

Cactus::presync_mode = "mixed-error"
CarpetX::poison_undefined_values = no

CarpetX::xmin = -13
CarpetX::ymin = -13
CarpetX::zmin = -13

CarpetX::xmax = 13
CarpetX::ymax = 13
CarpetX::zmax = 13

CarpetX::ncells_x = $ncells
CarpetX::ncells_y = $ncells
CarpetX::ncells_z = $ncells

CarpetX::boundary_x =  "neumann"
CarpetX::boundary_y =  "neumann"
CarpetX::boundary_z =  "neumann"
CarpetX::boundary_upper_x =  "neumann"
CarpetX::boundary_upper_y =  "neumann"
CarpetX::boundary_upper_z =  "neumann"

CarpetX::max_num_levels = $nlevels
CarpetX::regrid_every = 10000
CarpetX::blocking_factor_x = 1
CarpetX::blocking_factor_y = 1
CarpetX::blocking_factor_z = 1

# the regrid_error_threshold should be in the same units of the 
# parameter in comparison, see AsterX/src/estimate_error.cxx 
# for which parameter to use
CarpetX::regrid_error_threshold = 5.0e-5

CarpetX::prolongation_type = "ddf"
CarpetX::ghost_size = 3
CarpetX::dtfac = 0.35

#ADMBase::set_adm_variables_during_evolution = "yes"
ADMBase::initial_data       = "tov"
ADMBase::initial_lapse      = "tov"
ADMBase::initial_shift      = "tov"
ADMBase::initial_dtlapse    = "zero"
ADMBase::initial_dtshift    = "zero"


ActiveThorns = "TOVSolver"
TOVSolver::TOV_Rho_Central[0] = 1.28e-3
TOVSolver::TOV_Gamma          = 2.0
TOVSolver::TOV_K              = 100.0
TOVSolver::TOV_Cowling = yes


AsterSeeds::test_type = "3DTest"
AsterSeeds::test_case = "magTOV"
AsterSeeds::Afield_config = "internal dipole"
AsterSeeds::Ab = 10000.0
AsterSeeds::press_cut = 0.04
AsterSeeds::press_max = 1.638e-4
AsterSeeds::Avec_kappa = 2.0

#AsterSeeds::Afield_config = "external dipole"
#AsterSeeds::B0 = 1e-7
#AsterSeeds::r0 = 5.0

AsterX::rho_abs_min = 1e-13
AsterX::atmo_tol = 1e-3
AsterX::reconstruction_method = "wenoz"
AsterX::flux_type = "HLLE"
AsterX::max_iter = 30
AsterX::c2p_tol = 1e-8
AsterX::vector_potential_gauge = "algebraic"

EOSX::evol_eos_name = "IdealGas"
EOSX::gl_gamma = 2
EOSX::poly_gamma = 2.0
EOSX::poly_k = 100

ODESolvers::method = "RK4"

IO::out_dir = $parfile
IO::out_every =  20 #$ncells * 2 ** ($nlevels - 1) / 32
        
CarpetX::out_openpmd_vars = "
    HydroBase::rho
    HydroBase::Bvec
"

CarpetX::out_norm_vars = "HydroBase::rho"


Then, submit the simulation using the following command:

In [ ]:
%%bash
./simfactory/bin/sim create-submit MagnetizedTOV \
  --parfile=./par/MagnetizedTOV.par --procs=3 --num-threads=1 --ppn-used=3 --walltime=00:20:00

The above command creates and runs the simulation ```MagnetizedTOV```, using the configuration ```sim```. The data is saved in the directory ```./simulations/MagnetizedTOV```.



In [ ]:
# watch log output, following along as new output is produced
!./simfactory/bin/sim show-output --follow MagnetizedTOV

## 2. Steps to visualize simulation data

The 2D data can be saved in both Silo format (which can be visualised, for instance, via VisIt) and in OpenPMD format. 

For further info on Silo, please visit: https://wci.llnl.gov/simulation/computer-codes/silo

For further info about OpenPMD, please visit:

- Official website:  https://www.openpmd.org
- GitHub repository: https://github.com/openPMD
- Documentation:     https://openpmd-api.readthedocs.io

Now, let's go back to the home directory:

In [ ]:
cd ~/

Import all the required modules:

In [ ]:
%pip install --user celluloid

In [ ]:
import sys
sys.path.append("/usr/local/lib/python3.8/site-packages")

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from celluloid import Camera
import openpmd_api as io

Set the Matplotlib backend to `notebook`, not `inline`, since we'll want to animate some figures and the latter is not compatible with that

In [ ]:
%matplotlib notebook

Open a .bp file (ADIOS2 extension) as an OpenPMD **'series'**, which is a collection of **'iterations'**, each of which contains **'records'**, which are sets of either structured data --- **'meshes'** --- or unstructured data --- **'particles'**. 

AsterX only outputs mesh data. Each record has one or more **'components'**: for example, a record representing a scalar field only has one component, while a record representing a vector field has three.

In [ ]:
home = os.environ["HOME"]
series = io.Series(os.path.join(home, "simulations",
                                "MagnetizedTOV",
                                "output-0000","MagnetizedTOV",
                                "MagnetizedTOV.it%08T.bp5"), io.Access.read_only)

All iterations in our series have the same structure, i.e., they contain
the same records, since they all represent the same output, just at
different times. 

Here we define an empty Python nested dictionary whose
structure, once full, will be:

Iteration 0:
- Record 1:
  - Component 1: 3D data array
  - Component 2: 3D data array
  - Component 3: 3D data array
- Record 2:
  - Component 1: 3D data array
  - Component 2: 3D data array
  - Component 3: 3D data array
  
 [...]

Iteration 1:
- Record 1:
  - Component 1: 3D data array
  - Component 2: 3D data array
  - Component 3: 3D data array
- Record 2:
  - Component 1: 3D data array
  - Component 2: 3D data array
  - Component 3: 3D data array
  
 [...]

[...]

In [ ]:
iter_rec_comp_dict = {}

Print info, register data chunks and fill the above dictionary:

In [ ]:
for index in series.iterations:
    iteration = str(index)

    print("\nIteration " + iteration + ":")
    print("==============")

    # Allocate an empty dictionary associated to this iteration
    iter_rec_comp_dict[iteration] = {}

    i = series.iterations[index]

    for key in i.meshes:
        print("Components of record \"" + key + "\":")

        # Allocate an empty dictionary associated to this record. Notice that
        # 'record' is an OpenPMD mesh object, so it's better to use 'key'
        # instead of 'record' as a key in the dictionary ('record' could also be
        # used, but it makes accessing the key clumsy).
        record = i.meshes[key]
        iter_rec_comp_dict[iteration][key] = {}

        # Load each component of each record as a 'data chunk', i.e., an
        # allocated, but STILL INVALID, NumPy array. Later we will flush all
        # chunks (i.e., basically, fill the NumPy arrays) at once: this leads
        # to better I/O performance compared to flushing a large number of
        # small chunks. That's why we bothered creating the nested dictionary:
        # in this way, we can access the valid NumPy arrays for plotting
        # without having to flush each single chunk.
        # *IMPORTANT*: DO NOT access data chunks until flushing has happened!
        for component in record:
            print("    > " + component)  # 'component' is a string
            iter_rec_comp_dict[iteration][key][component] = record[component].load_chunk()  # *INVALID* 3D NumPy array

        print("")

Flush all registered data chunks, which are now **VALID** 3D NumPy arrays:

In [ ]:
series.flush()

Visualize a 2D movie of the mass density on the xz plane:

In [ ]:
# Select the desired record and component to plot
record    = "hydrobase_rho_lev00"  # "carpetx_regrid_error_lev00", "hydrobase_eps_lev00", "hydrobase_press_lev00", "hydrobase_rho_lev00", "hydrobase_vel_lev00"
component = "hydrobase_rho"  # "carpetx_regrid_error", "hydrobase_eps", "hydrobase_press", "hydrobase_rho", "hydrobase_velx", "hydrobase_vely", "hydrobase_velz"

# Set up the axes for the plot and the colorbar
fig    = plt.figure(figsize = [4.8, 4])
axplot = fig.add_axes([0.12, 0.14, 0.72, 0.74])
axclb  = fig.add_axes([0.85, 0.14, 0.02, 0.74])
#axplot = fig.add_axes([0.12, 0.14, 0.7, 0.7])
#axclb  = fig.add_axes([0.8, 0.16, 0.05, 0.65])


# Set title and labels
axplot.set_title("Rest-mass density", fontsize = 10., fontweight = "bold", color = "midnightblue")
axplot.set_xlabel("x", fontsize = 7.)
axplot.set_ylabel("z", fontsize = 7.)
axplot.tick_params(labelsize=7)
axplot.xaxis.set_major_locator(plt.MaxNLocator(5))
axplot.yaxis.set_major_locator(plt.MaxNLocator(5))


# Initialize the camera
camera = Camera(fig)

# Print frames
for iteration in iter_rec_comp_dict:
    # Retrieve the 3D array containing the data
    array3D = iter_rec_comp_dict[iteration][record][component]
    
    # Plot on the (x, y) plane at the half-way value of z
    # Notice that the 3D array is stored in (z, y, x) order
    z_index = int(array3D.shape[0]/2)
    x0     = np.linspace(-14, 14, array3D.shape[2])
    y0     = np.linspace(-14, 14, array3D.shape[1])
    log10_rho = np.log10(array3D[z_index, :, :])
    image   = axplot.pcolormesh(x0, y0, log10_rho,
                                cmap = "inferno", vmin = -5.8 , vmax = -2.8)
    axplot.set_xlim(xmin=-11, xmax=11)
    axplot.set_ylim(ymin=-11, ymax=11)
    # Set up the colorbar
    axclb.tick_params(labelsize=7.0)
    fig.colorbar(image, cax = axclb, extend = "neither")
    
    # Print the current iteration
    axplot.text(5, 9, "Iteration " + iteration,
             fontsize = 8., fontweight = "bold", color = "white")

    # Take a snapshot of the figure at this iteration (needed later for the animation)
    camera.snap()

In [ ]:
from IPython.display import HTML
animation = camera.animate()
HTML(animation.to_html5_video())

You may delete the simulation directory via the following:


In [ ]:
%cd ~/
%rm -r ./simulations/MagnetizedTOV